In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

import acquire
import explore
import prepare

## Acquire & Summarize

#### 1.) Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.
    - a.)Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
    - b.)Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction.
    - c.) Only include properties that include a latitude and longitude value.

-----------------------------

In [2]:
zillow_df = acquire.zillow_data_all()
zillow_df.head(10)
zillow_df 

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,buildingclasstypeid,buildingclassdesc,heatingorsystemtypeid,heatingorsystemdesc,buildingclasstypeid,buildingclassdesc,storytypeid,storydesc,typeconstructiontypeid,typeconstructiondesc
0,1727539,14297519,nan,nan,nan,3.50,4.00,nan,nan,3.50,...,nan,None,nan,None,nan,None,nan,None,nan,None
1,1387261,17052889,nan,nan,nan,1.00,2.00,nan,nan,1.00,...,nan,None,nan,None,nan,None,nan,None,nan,None
2,11677,14186244,nan,nan,nan,2.00,3.00,nan,nan,2.00,...,nan,None,nan,None,nan,None,nan,None,nan,None
3,2288172,12177905,nan,nan,nan,3.00,4.00,nan,8.00,3.00,...,nan,None,2.00,Central,nan,None,nan,None,nan,None
4,1970746,10887214,1.00,nan,nan,3.00,3.00,nan,8.00,3.00,...,nan,None,2.00,Central,nan,None,nan,None,nan,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77574,2864704,10833991,1.00,nan,nan,3.00,3.00,nan,8.00,3.00,...,nan,None,2.00,Central,nan,None,nan,None,nan,None
77575,673515,11000655,nan,nan,nan,2.00,2.00,nan,6.00,2.00,...,nan,None,2.00,Central,nan,None,nan,None,nan,None
77576,2968375,17239384,nan,nan,nan,2.00,4.00,nan,nan,2.00,...,nan,None,nan,None,nan,None,nan,None,nan,None
77577,1843709,12773139,1.00,nan,nan,1.00,3.00,nan,4.00,1.00,...,nan,None,2.00,Central,nan,None,nan,None,nan,None


In [3]:
#initial shape of dataframe
zillow_df.shape

(77579, 79)

In [4]:
## making minor changes before prep.
zillow_df = zillow_df.loc[:, ~zillow_df.columns.duplicated()]
zillow_df.shape

(77579, 68)

In [6]:
z_df = zillow_df.sort_values('transactiondate').groupby('parcelid').last()
z_df.head(10)

,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,censustractandblock,logerror,transactiondate,propertylandusedesc,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,storydesc,typeconstructiondesc
parcelid,,,,,,,,,,,,,,,,,,,,,
10711855,1087254,nan,nan,nan,2.00,3.00,nan,8.00,2.00,nan,...,"60,371,132,321,007.00",-0.01,2017-07-07,Single Family Residential,None,None,None,Central,None,None
10711877,1072280,1.00,nan,nan,2.00,4.00,nan,8.00,2.00,nan,...,"60,371,132,321,007.00",0.02,2017-08-29,Single Family Residential,Central,None,None,Central,None,None
10711888,1340933,1.00,nan,nan,2.00,4.00,nan,8.00,2.00,nan,...,"60,371,132,321,007.00",0.08,2017-04-04,Single Family Residential,Central,None,None,Central,None,None
10711910,1878109,nan,nan,nan,2.00,3.00,nan,8.00,2.00,nan,...,"60,371,132,321,008.00",-0.04,2017-03-17,Single Family Residential,None,None,None,Central,None,None
10711923,2190858,nan,nan,nan,2.00,4.00,nan,8.00,2.00,nan,...,"60,371,132,321,008.00",-0.01,2017-03-24,Single Family Residential,None,None,None,Central,None,None
10711945,281974,1.00,nan,nan,2.00,3.00,nan,8.00,2.00,nan,...,"60,371,132,321,016.00",0.00,2017-01-30,Single Family Residential,Central,None,None,Central,None,None
10711956,2981006,1.00,nan,nan,3.00,3.00,nan,8.00,3.00,nan,...,"60,371,132,321,009.00",0.02,2017-07-03,Single Family Residential,Central,None,None,Central,None,None
10711995,2548580,1.00,nan,nan,2.00,4.00,nan,8.00,2.00,nan,...,"60,371,132,321,003.00",0.04,2017-03-09,Single Family Residential,Central,None,None,Central,None,None
10712005,1699185,1.00,nan,nan,2.00,3.00,nan,8.00,2.00,nan,...,"60,371,132,321,005.00",0.01,2017-08-01,Single Family Residential,Central,None,None,Central,None,None


#### 2.) Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [12]:
prepare.summarize(zillow_df)

Dataframe head: 
        id  parcelid  airconditioningtypeid  architecturalstyletypeid  \
0  1727539  14297519                    nan                       nan   
1  1387261  17052889                    nan                       nan   
2    11677  14186244                    nan                       nan   
3  2288172  12177905                    nan                       nan   
4  1970746  10887214                   1.00                       nan   
5  1447245  17143294                    nan                       nan   
6   781532  12095076                   1.00                       nan   
7   870991  12069064                    nan                       nan   
8  1246926  12790562                    nan                       nan   
9  1585097  11542646                    nan                       nan   

          basementsqft          bathroomcnt           bedroomcnt  \
0                  nan                 3.50                 4.00   
1                  nan                 1.00

#### 3.) Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [25]:
explore.miss_dup_values(zillow_df)

Your selected dataframe has 79 columns.
There are 60 columns that have missing values.
  
** There are 0 duplicate rows that represents 0.0% of total Values**


,Missing Values,% of Total Values
buildingclasstypeid,77564,100.00
buildingclassdesc,77564,100.00
buildingclasstypeid,77564,100.00
buildingclasstypeid,77564,100.00
buildingclassdesc,77564,100.00
finishedsquarefeet13,77537,99.90
storytypeid,77529,99.90
storytypeid,77529,99.90
storydesc,77529,99.90
basementsqft,77529,99.90


# Prepare:

-----------------------------